# Rankingデータの作成
　前節までの順位を計算し, csv出力を行う.

## import

In [1]:
import pandas as pd

## Setting

In [2]:
# Path
input_path = "../input_data/"

# Set Display Max Columns
pd.set_option("display.max_columns", 50)

## Read Data

In [3]:
train = pd.read_csv(input_path + "jleague/train.csv", sep=",", header=0, quotechar="\"")
train_add = pd.read_csv(input_path + "jleague/train_add.csv", sep=",", header=0, quotechar="\"")

test = pd.read_csv(input_path + "jleague/test.csv", sep=",", header=0, quotechar="\"")
test_add = pd.read_csv(input_path + "jleague/2014_add.csv", sep=",", header=0, quotechar="\"")

condition = pd.read_csv(input_path + "jleague/condition.csv", sep=",", header=0, quotechar="\"")
condition_add = pd.read_csv(input_path + "jleague/condition_add.csv", sep=",", header=0, quotechar="\"")

stadium = pd.read_csv(input_path + "jleague/stadium.csv", sep=",", header=0, quotechar="\"")

## Merge

In [4]:
condition_all = pd.concat([condition, condition_add], axis=0)

In [5]:
train_all = pd.concat([train, train_add], axis=0)
train_all = pd.merge(train_all, condition_all, how = "inner", on = "id")

In [6]:
test_all = pd.concat([test, test_add], axis=0)
test_all = pd.merge(test_all, condition_all, how = "inner", on = "id")

## Make Ranking
### Function

In [7]:
# 節数を数値に変換する関数
def match_num_fun(x):
    y = x.split("節")[0].replace("第", "")
    return int(y)

# 勝ち点関数
def home_win_fun(row):
    if row["home_score"] > row["away_score"]:
        y = 3
    elif row["home_score"] == row["away_score"]:
        y = 1
    else:
        y= 0
    return y

def away_win_fun(row):
    if row["away_score"] > row["home_score"]:
        y = 3
    elif row["away_score"] == row["home_score"]:
        y = 1
    else:
        y= 0
    return y

### Score Table

In [8]:
score_table = pd.concat([train_all, test_all], axis=0, sort=False).reset_index(drop=True)

# 節数を数値に変換
score_table["match_num"] = score_table["match"].apply(match_num_fun)

# 勝ち点
score_table["home_point"] = score_table.apply(home_win_fun, axis=1)
score_table["away_point"] = score_table.apply(away_win_fun, axis=1)

#  得失点差
score_table["home_goal_diff"] = score_table["home_score"] - score_table["away_score"]
score_table["away_goal_diff"] = score_table["away_score"] - score_table["home_score"]

score_table = score_table[[
    "id", "year", "stage", "match_num", "home", "away",
    "home_point", "away_point", "home_goal_diff", "away_goal_diff",
    "home_score", "away_score"]]

In [9]:
# 欠損確認
score_table.isnull().any()

id                False
year              False
stage             False
match_num         False
home              False
away              False
home_point        False
away_point        False
home_goal_diff    False
away_goal_diff    False
home_score        False
away_score        False
dtype: bool

### Calculate Ranking
　チーム, 試合ごとに前節までの各累計値を計算して, 順位を計算する.

In [10]:
# home
home_s = score_table.rename(
    columns={
        "home": "team", "home_point": "point", "home_goal_diff": "goal_diff", "home_score": "score"}, inplace=False)
home_s = home_s[["id", "stage", "year", "match_num", "team", "point", "goal_diff", "score"]].copy()

# away
away_s = score_table.rename(
    columns={
        "away": "team", "away_point": "point", "away_goal_diff": "goal_diff", "away_score": "score"}, inplace=False)
away_s = away_s[["id", "stage", "year", "match_num", "team", "point", "goal_diff", "score"]].copy()

# concat
score_table_all = pd.concat([home_s, away_s], axis=0).reset_index(drop=True)

In [11]:
score_table_all.shape

(4608, 8)

　このとき, 前節までの勝ち点, 得失点差, 総得点の累積和を計算し, 勝ち点の高い順に順位を作成. 同順位の場合は, 得失点差で順位をつける. まだ同順位の場合は総得点数で順位をつける.  
※正確な順位の付け方は下記URLの「年間順位の決定」を参照  
https://www.jleague.jp/standings/j1/

In [12]:
# チームごとに時系列に並べる
ranking_table_all = score_table_all.sort_values(["team","year", "match_num"]).reset_index(drop=True)

# チーム, 年ごとに前節の結果にずらす
ranking_table_all = pd.concat(
    [
        ranking_table_all[["id", "stage", "year", "match_num", "team"]],
        # 第1節はどの値も0とする
        ranking_table_all.groupby(["team", "year"])[["point", "goal_diff", "score"]].shift(periods=1).fillna(0)
    ], axis=1)


# チーム, 年ごとに前節までの累計和を計算(第1節はどの値も0とする)
ranking_table_all["point_cumsum"] = ranking_table_all.groupby(["team", "year"])["point"].cumsum()
ranking_table_all["goal_diff_cumsum"] = ranking_table_all.groupby(["team", "year"])["goal_diff"].cumsum()
ranking_table_all["score_cumsum"] = ranking_table_all.groupby(["team", "year"])["score"].cumsum()

In [13]:
# stage, 年, 節ごとに各値での順位を計算
ranking_table_all["rank_1"] = ranking_table_all.groupby(["stage", "year", "match_num"])["point_cumsum"].rank(ascending=False, method="min")
ranking_table_all["rank_2"] = ranking_table_all.groupby(["stage", "year", "match_num"])["goal_diff_cumsum"].rank(ascending=False, method="min")
ranking_table_all["rank_3"] = ranking_table_all.groupby(["stage", "year", "match_num"])["score_cumsum"].rank(ascending=False, method="min")

# チーム名数は42なので, 2桁ずつずらして足す
ranking_table_all["rank_score"] = ranking_table_all["rank_1"] * 10000 + ranking_table_all["rank_2"] * 100 + ranking_table_all["rank_3"]

# stage, 年, 節ごとに最終的な順位を計算(rank_scoreの小さい順)
ranking_table_all["rank"] = ranking_table_all.groupby(["stage", "year", "match_num"])["rank_score"].rank(ascending=True, method="min")
ranking_table_all["rank"] = ranking_table_all["rank"].astype(int)

# 必要な変数に絞る(紐づけはid, teamで出来るが, 確認用にstage, year, match_numは残す)
ranking_table_all = ranking_table_all[["id","stage",  "year", "match_num", "team", "rank"]]

In [14]:
ranking_table_all.head()

,id,stage,year,match_num,team,rank
0,14305,Ｊ２,2012,1,アビスパ福岡,1
1,14313,Ｊ２,2012,2,アビスパ福岡,5
2,14327,Ｊ２,2012,3,アビスパ福岡,4
3,14336,Ｊ２,2012,4,アビスパ福岡,9
4,14348,Ｊ２,2012,5,アビスパ福岡,9


In [15]:
ranking_table_all.shape

(4608, 6)

## Output

In [16]:
ranking_table_all.to_csv(
    "../output_data/ranking_table_all.csv",  sep=",", index=False, header=True)